In [0]:
!pip install tqdm
dbutils.library.restartPython()

In [0]:
from pyspark.sql.functions import current_timestamp
from tqdm import tqdm

# Salva o nome dos arquivos automaticamente
files = [(f.name) for f in dbutils.fs.ls("/Volumes/c_bikeshop/raw/raw_files/")]

for file in tqdm(files):
    path = f'dbfs:/Volumes/c_bikeshop/raw/raw_files/' 
    
    # Lê os arquivos csv e cria um dataframe
    df = (
        spark.read
        .option('header', True)
        .option('inferSchema',True)
        .csv(path)        
        .withColumn('_ingest_ts', current_timestamp())
    )

    #Cria tabelas no schema bronze 
    (
        df.write
        .format('delta')
        .mode('overwrite')
        .saveAsTable(f'c_bikeshop.bronze.{file.replace(".csv", "")}')
    )
    
    print(f'Tabela bronze {file} criada com sucesso!')

